In [1]:
!pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 11.0 MB/s eta 0:00:00


In [2]:
import json

# Define the path to the JSON file
path_to_json = '/kaggle/input/metu-web-dataset/metu_dataset.json'

# Load your JSON data from the file
with open(path_to_json, 'r') as file:
    data = json.load(file)

# Extract URL and content into a new list of dictionaries
extracted_data = [
    {
        "URL": entry["URL"],
        "CONTENT": entry["content"]
    }
    for entry in data
]


In [3]:
from sentence_transformers import SentenceTransformer

model_name = 'distiluse-base-multilingual-cased-v1'

# Initialize the SentenceTransformer model
model = SentenceTransformer(f'sentence-transformers/{model_name}')

# Create a list of sentences from the extracted data
sentences = [entry["CONTENT"] for entry in extracted_data]

# Generate embeddings for the sentences
embeddings = model.encode(sentences)


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.47k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/556 [00:00<?, ?B/s]

...

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

Batches:   0%|          | 0/843 [00:00<?, ?it/s]

In [4]:
import pandas as pd
# Create a DataFrame to store DOCNO and corresponding embeddings
embeddings_df = pd.DataFrame(embeddings)
embeddings_df.insert(0, 'URL', [entry["URL"] for entry in extracted_data])

# Save the DataFrame as a CSV file
embeddings_df.to_csv(f'{model_name}_METU_embeddings.csv', index=False)

# Search

In [8]:
!pip install faiss-cpu
!pip install -U sentence-transformers

/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 70.7 MB/s eta 0:00:00:00:0100:01


In [9]:
import pandas as pd
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

# Load the CSV file containing document embeddings
df = pd.read_csv('/kaggle/input/distiluse-base-multilingual-cased-v1-metu-em/distiluse-base-multilingual-cased-v1_METU_embeddings.csv')

# Assuming the first column is DOCNO and the rest are embeddings
docnos = df['URL'].values  # Extract document numbers
embeddings = df.drop(columns=['URL']).values.astype('float32')  # Extract embeddings and convert to float32

# Initialize FAISS index
dim = embeddings.shape[1]  # Dimensionality of the embeddings
#index = faiss.IndexFlatL2(dim)  # Using L2 distance for similarity search
index = faiss.IndexFlatIP(dim)
# Add embeddings to the FAISS index
index.add(embeddings)

model_name = 'distiluse-base-multilingual-cased-v1'

# Initialize the SentenceTransformer model for query embedding
model = SentenceTransformer(f'sentence-transformers/{model_name}')


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [11]:

# Function to perform similarity search
def search_similar_documents(query, k=5):
    # Generate embedding for the query using SentenceTransformer
    query_embedding = model.encode([query]).astype('float32')
    
    # Perform the search on the FAISS index
    distances, indices = index.search(query_embedding, k)
    
    # Retrieve the corresponding document numbers and distances
    similar_docs = [(docnos[idx], distances[0][i]) for i, idx in enumerate(indices[0])]
    return similar_docs

# Example usage of the similarity search function
query_text = "METU history"
similar_documents = search_similar_documents(query_text, k=20)

# Print results
for docno, distance in similar_documents:
    print(f'Document No: {docno}, Distance: {distance}')


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Document No: https://bidb.metu.edu.tr/en/history, Distance: 0.4443143606185913
Document No: https://hist.metu.edu.tr/en, Distance: 0.434567928314209
Document No: https://hist.metu.edu.tr/, Distance: 0.434567928314209
Document No: https://fedu.metu.edu.tr/en/history, Distance: 0.3764258623123169
Document No: https://me.metu.edu.tr/history, Distance: 0.3678094148635864
Document No: http://www.metu.edu.tr/history, Distance: 0.3658742308616638
Document No: https://id.metu.edu.tr/en/history, Distance: 0.3650658428668976
Document No: https://hist.metu.edu.tr/en/history-resources, Distance: 0.35718339681625366
Document No: https://hist.metu.edu.tr/en/department-history-0, Distance: 0.3554767668247223
Document No: https://mine.metu.edu.tr/en/node/97, Distance: 0.3507682681083679
Document No: https://mine.metu.edu.tr/en/department-history, Distance: 0.3507682681083679
Document No: https://ba.metu.edu.tr/en/system/files/odtu_2024_tanitim_fuari_genel_programi.pdf, Distance: 0.3474454879760742
Doc